In [2]:
import pandas as pd

# CSV 파일을 DataFrame으로 읽어오기
df = pd.read_csv('stk_optOrdProgress_20241226_170039.csv', encoding='CP949')
# df = df.drop(columns=['20241226', 'Unnamed: 3'])
df = df.iloc[:, :-2]
df = df[df['현재고'] < 0]
df['현재고'] = 0

# DataFrame 출력
df

,바코드번호,현재고
7891,1-1377620002,0
7902,1-1381120002,0
8296,1-1430220001,0
8297,1-1430220003,0
8299,1-1430220005,0
8436,1-1315120002,0
8438,1-1315120004,0
8439,1-1315120005,0
8583,1-1461320000,0
8878,1-1488420001,0


In [4]:
from datetime import datetime

current_time = datetime.now().strftime('%Y%m%d_%H%M%S')
df.to_csv(f'option_modify_form_{current_time}.csv', index=False, encoding='CP949')

In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import TimeoutException, UnexpectedAlertPresentException, NoAlertPresentException
from selenium.webdriver.common.alert import Alert
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd
import os
from datetime import datetime
from dotenv import load_dotenv
from pathlib import Path

In [4]:
# .env 파일 로드
load_dotenv()

DOWNLOAD_TIMEOUT = 60  # 다운로드 대기 시간

# Chrome 옵션 설정
options = webdriver.ChromeOptions()
download_prefs = {
    "download.default_directory": os.path.expanduser(os.getenv('DOWNLOAD_PATH')),
    "download.prompt_for_download": False,
    "download.directory_upgrade": True,
    "safebrowsing.enabled": True
}
options.add_experimental_option("prefs", download_prefs)

driver = webdriver.Chrome(options=options)
wait = WebDriverWait(driver, 2)

try:
    # 페이지 열기 및 로그인
    driver.get(os.getenv('BASE_URL'))
    time.sleep(2)

    # 로그인 정보 입력
    userid_field = driver.find_element(By.ID, "userid")
    password_field = driver.find_element(By.ID, "passwd")
    
    userid_field.clear()
    password_field.clear()
    
    userid_field.send_keys(os.getenv("SELLMATE_ID"))
    password_field.send_keys(os.getenv("SELLMATE_PW"))
    
    driver.find_element(By.CLASS_NAME, "login_btn").click()
    time.sleep(2)

    # 재고 보고서 페이지 접근
    driver.get(f"{os.getenv('BASE_URL')}/stock/progressRpt.asp")
    time.sleep(2)

    driver.find_element(By.ID, "date_today").click()
    driver.find_element(By.ID, "search-btn").click()
    time.sleep(2)

    excel_dropdown = driver.find_element(By.ID, "excel_dropdown")
    excel_dropdown.click()
    time.sleep(1)

    driver.find_element(By.XPATH, "//a[contains(text(), 'XLS양식설정')]").click()

    main_window = driver.current_window_handle  # 현재 창 핸들 저장
    
    for handle in driver.window_handles:
        if handle != main_window:
            driver.switch_to.window(handle)
            time.sleep(3)  # 잠시 대기
            
            # Alert이 있는 경우 처리
            try:
                while True:
                    try:
                        alert = driver.switch_to.alert
                        alert.accept()  # Alert 확인 클릭
                        time.sleep(1)  # 잠시 대기
                    except :
                        break
            except Exception as e:  # 모든 예외를 포착
                if isinstance(e, UnexpectedAlertPresentException): # UnexpectedAlertPresentException 처리 코드
                    print("예기치 않은 Alert가 발생했습니다.")
                    raise e
                
                elif isinstance(e, NoAlertPresentException): # NoAlertPresentException 처리 코드
                    print("Alert가 없습니다.")
                    raise e
                    
                else: # 다른 예외 처리
                    print(f"처리되지 않은 예외 발생: {e}")
                    raise e

            # "음수재고 0 만들기" 선택
            template_select = driver.find_element(By.CLASS_NAME, "xls-template-options")
            for option in template_select.find_elements(By.TAG_NAME, "option"):
                if option.text == "음수재고 0 만들기":
                    option.click()
                    break
            
            driver.close()
            break

    driver.switch_to.window(main_window)  # 메인 창으로 돌아가기

    excel_dropdown.click()
    time.sleep(1)

    # '엑셀다운로드' 항목 선택
    excel_download_option = wait.until(
        EC.element_to_be_clickable((By.XPATH, "//li[@role='presentation' and @onclick='popXls();']/a"))
    )
    excel_download_option.click()

    excel_file_name = ""

    # 새 창에서 다운로드 버튼 클릭
    for handle in driver.window_handles:
        if handle != main_window:
            driver.switch_to.window(handle)
            try:
                # 다운로드 버튼이 있는 테이블 행 추출
                row = WebDriverWait(driver, 30).until(
                    EC.presence_of_element_located(
                        (By.CSS_SELECTOR, "table.origin_table tbody:first-of-type tr:first-of-type")
                    )
                )

                # 다운로드 버튼 클릭
                download_button = WebDriverWait(row, 10).until(
                    EC.presence_of_element_located((By.CSS_SELECTOR, "td:last-child button.btn-success"))
                )

                # 완료시간 추출
                completed_time_cell = row.find_element(By.CSS_SELECTOR, "td:nth-child(5)")
                completed_time = datetime.strptime(completed_time_cell.text, "%Y-%m-%d %H:%M:%S")
                excel_file_name = f"stk_optOrdProgress_{completed_time.strftime('%Y%m%d_%H%M%S')}.csv"

                # 다운로드 버튼 클릭
                download_button.click()

                # 다운로드 완료 대기 개선
                downloaded_file_path = os.path.join(os.getenv('DOWNLOAD_PATH'), excel_file_name)
                download_path = Path(downloaded_file_path)
                wait_start = time.time()
                
                while not download_path.exists() or download_path.stat().st_size == 0:
                    if time.time() - wait_start > DOWNLOAD_TIMEOUT:
                        raise TimeoutException(f"파일 다운로드가 {DOWNLOAD_TIMEOUT}초 내에 완료되지 않았습니다.")
                    time.sleep(0.5)
                print(f"다운로드 완료: {downloaded_file_path}")

            except TimeoutException as e:
                print(f"오류 발생: {e}")
                raise e
            
            driver.close()
            break

    # CSV 파일 처리
    time.sleep(5)
    csv_path = os.path.join(os.getenv('DOWNLOAD_PATH'), excel_file_name)

    # 파일 존재 확인
    if not os.path.exists(csv_path):
        raise FileNotFoundError(f"CSV 파일을 찾을 수 없습니다: {csv_path}")
    
    try:
        index_df = pd.read_csv(csv_path, encoding='CP949')
        index_df = index_df.iloc[:, :-2]
        index_df = index_df[index_df['현재고'] < 0]
        index_df['현재고'] = 0

        # 결과 저장
        current_time = datetime.now().strftime('%Y%m%d_%H%M%S')
        output_filename = f'option_modify_form_{current_time}.csv'
        index_df.to_csv(output_filename, index=False, encoding='CP949')
        print(f"\n결과 저장 완료: {output_filename}")

    except Exception as e:
        print(f"CSV 파일 처리 중 오류 발생: {e}")
        raise  # 에러 전파하여 디버깅 용이하게

    driver.switch_to.window(main_window)  # 메인 창으로 돌아가기

    # 상품 옵션 일괄수정 페이지로 이동
    driver.get(f"{os.getenv('BASE_URL')}/stock/opt_csv_edit.asp")

    # 파일 입력 필드가 존재할 때까지 대기 (최대 10초)
    file_input = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.NAME, "pro_filename"))
    )

    # 생성된 CSV 파일의 절대 경로를 send_keys로 전송
    file_input.send_keys(os.path.abspath(output_filename))

    # 파일 전송 버튼이 클릭 가능할 때까지 대기 (최대 10초)
    submit_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, "//button[contains(text(), '파일전송')]"))
    )
    submit_button.click()

finally:
    # 브라우저 닫기
    driver.quit()

다운로드 완료: C:\Users\hyunb\Downloads\stk_optOrdProgress_20241230_150703.csv

결과 저장 완료: option_modify_form_20241230_150709.csv
